In [107]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy
/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy
/kaggle/input/bajlecgrausayegh/keras/default/1/config.json
/kaggle/input/bajlecgrausayegh/keras/default/1/metadata.json
/kaggle/input/bajlecgrausayegh/keras/default/1/model.weights.h5


In [108]:
from matplotlib import pyplot as plt
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [109]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Activation
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import models
from tensorflow.keras import layers, models, regularizers, constraints

In [110]:
x_train = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_train.npy")
x_test = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/x_test.npy")
y_train_coarse = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_coarse.npy")
y_train_fine = np.load("/kaggle/input/dl-itba-cifar-100-2024-q-1/y_train_fine.npy")

In [111]:
from sklearn.model_selection import train_test_split

In [112]:
import pickle
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/fine_label_names.pck", "rb") as f:
    labels_fine = pickle.load(f)
with open("/kaggle/input/dl-itba-cifar-100-2024-q-1/coarse_label_names.pck", "rb") as f:
    labels_coarse = pickle.load(f)

In [113]:
y_train_fine.shape

(50000,)

In [114]:
x_train.shape

(50000, 32, 32, 3)

In [115]:
x_train,x_val, y_train, y_val = train_test_split(x_train, y_train_fine, test_size=0.1, stratify=y_train_fine)

In [116]:
y_val.shape

(5000,)

In [117]:
x_val.shape

(5000, 32, 32, 3)

In [118]:
x_train_horizontal_flip= x_train[:,:,::-1,:]
#x_train_vertical_flip = x_train[:,::-1,:,:]
#x_train_vh_flip =x_train[:, ::-1, ::-1, :]

#x_train_dup = np.concatenate([x_train,x_train_horizontal_flip,x_train_vertical_flip,x_train_vh_flip],axis=0)
x_train_dup = np.concatenate([x_train,x_train_horizontal_flip],axis=0)
#y_train_dup = np.concatenate([y_train, y_train,y_train, y_train], axis=0)
y_train_dup = np.concatenate([y_train, y_train], axis=0)

x_train_bis = x_train_dup.copy()
x_train_bis_bis = x_train_dup.copy()

#desplazamientos right and left 
x_train_bis[:,:,1:,:] =x_train_dup[:,:,:-1,:]
x_train_bis_bis[:,:,:-1,:] =x_train_dup[:,:,1:,:]

x_train_dup = np.concatenate([x_train_dup, x_train_bis, x_train_bis_bis],axis=0)
y_train_dup = np.concatenate([y_train_dup, y_train_dup, y_train_dup], axis=0)


In [119]:
print(f'Original dataset size: {len(x_train)}')
print(f'Augmented dataset size: {len(x_train_dup)}')

Original dataset size: 45000
Augmented dataset size: 270000


In [120]:
x_train_dup.shape, y_train_dup.shape

((270000, 32, 32, 3), (270000,))

In [121]:
import keras

In [152]:
def create_model(input_shape=(32, 32, 3),l2_lambda=0.001):
    # Input layer for images
    inputs = layers.Input(shape=input_shape)
    
    # Flattening layer
    x = layers.Flatten()(inputs)
    
    # Normalization layer
    x = layers.Lambda(lambda x: x/np.float16(255.0))(x)
    
    # Dense layers with L1 regularization
    n = 12
    for i in range(2):
        x = layers.Dense(2**n,
                         kernel_regularizer=regularizers.L2(l2_lambda))(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
       
    # Output layer with softmax for 100 classes
    fine_output = layers.Dense(100, activation='softmax', name='fine_output')(x)   
  
    # Create model
    model = models.Model(inputs=inputs, outputs=[fine_output], name=f"mlp_{2**i}")
    
    return model

In [153]:
model = create_model()
model.compile(
    optimizer=Adam(),
    loss={'fine_output': 'sparse_categorical_crossentropy'},
    metrics={'fine_output': 'accuracy' }
             )

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True,verbose= 1)
tb = TensorBoard(log_dir="logs")
# Display the model summary
model.summary()

Model: "mlp_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_30 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_30 (Flatten)            │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda_30 (Lambda)              │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_103 (Dense)               │ (None, 4096)           │    12,587,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_103         │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_73 (ReLU)                 │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_104         │ (None, 4096)           │        16,384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_74 (ReLU)                 │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fine_output (Dense)             │ (None, 100)            │       409,700 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,810,788 (113.72 MB)

 Trainable params: 29,794,404 (113.66 MB)

 Non-trainable params: 16,384 (64.00 KB)

In [ ]:
# Fit the model
history = model.fit(x=x_train_dup,   
                    y={'fine_output': y_train_dup},
                    validation_data=(x_val, {'fine_output': y_val}),
                    epochs=50,                        
                    batch_size=512,                      
                    callbacks=[tb, early_stopping])

Epoch 1/50
528/528 ━━━━━━━━━━━━━━━━━━━━ 23s 34ms/step - accuracy: 0.1526 - loss: 7.0394 - val_accuracy: 0.0660 - val_loss: 5.8258
Epoch 2/50
346/528 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.2231 - loss: 3.9430

In [125]:
# Save the model after 25 epochs
#model.save("accuracy0.2_manylayers_batch_norm2.keras")

In [130]:
predictions = model.predict(x_test).argmax(axis=1)
df = pd.DataFrame(predictions, columns=["Label"])

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [131]:
df

,Label
0,17
1,31
2,15
3,51
4,49
...,...
9995,88
9996,21
9997,54
9998,80


In [132]:
df.index.name = "Id"

In [133]:
df.to_csv("submission.csv")